## Autors Agris Pudāns, ap08426

### Pamats Jupyter Notebook veidošanai.

1) Koda izpildi kontrolē viens mainīgais - slēdzis  
2) Atkarībā no slēdža stāvokļa, tiek izpildīts tikai modeļa apmācībai vai tikai modeļa pārbaudei nepieciešamais kods  
3) Tiek ielādēti tikai tie moduļi un dati, kas nepieciešami konkrētās darbības izpildei  
4) Ja, pārbaudot kodu, nepieciešams darīt vairāk nekā nomainīt slēdzi un norādīt testa datu kopas ielādes URL, darbs nav korekti pabeigts, notestēts, utt  
5) Pabeigta darba kodam jāstrādā bez aizķeršanās, palaižot to ar Run All  
6) Pēc iespējas visu darbu darīt ar klasēm un funkcijām

### Kontroles bloks
Definē slēdzi un satur saites uz datiem un trenēto modeli

In [1]:
# True - ielādē treniņa datu kopu un uz tās apmāca jaunu modeli, kuru saglabā
# False - ielādē jau apmācītu modeli un atsevišķu datu kopu, uz kuras modeli pārbaudīt
TRAIN = False

# Norādam apmācīta modeļa ielādes URL. Nomainīt.
MODEL_URL = 'https://raw.githubusercontent.com/achenlv/ap08426-random/refs/heads/main/iris_model.pth'
# Norādam treniņa datu kopas ielādes URL. Nomainīt.
TRAIN_DATA_URL = 'https://raw.githubusercontent.com/achenlv/ap08426-random/refs/heads/main/Iris_train.csv'
# Norādam testa datu kopas ielādes URL. Testējot kodu, pagaidām ir validācijas datu kopa, taču pārbaudot darbu pasniedzējs nomainīs uz nodalītu vērtēšanas kopu
TEST_DATA_URL = 'https://raw.githubusercontent.com/achenlv/ap08426-random/refs/heads/main/Iris_validate.csv'

### Moduļu importēšana
Importē nepieciešamos moduļus.

In [2]:
# os modulis nepieciešams darbam ar failiem
import os
# torch modulis nodrošina MI apmācībai un lietošanai nepieciešamās funkcijas
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
# numpy modulis nodrošina dažādas matemātiskās funkcijas
import numpy as np
# Pandas modulis tabulveida datu apstrādei
import pandas as pd
# requests modulis tīkla pieprasījumiem
import requests
from io import BytesIO
import warnings
warnings.filterwarnings('ignore')


### Klašu un funkciju definēšana
Ieteicams visu definēt koda sākumā.  
Komentējiet pašas funkcijas mērķi un galvenās darbības tās iekšienē.  
Ja nepieciešams, definējiet papildus klases un funkcijas.

In [3]:
# Definē klasi IrisDataset, kas saturēs datus neironu tīklam saprotamā formātā
class IrisDataset(Dataset):
    def __init__(self, features, labels):
        """
        Inicializē datu kopu ar pazīmēm un etiķetēm
        
        Parametri:
        features (torch.Tensor): Pazīmju masīvs
        labels (torch.Tensor): Etiķešu masīvs
        """
        self.features = features
        self.labels = labels
    
    def __len__(self):
        """
        Atgriež datu kopas garumu
        """
        return len(self.features)
    
    def __getitem__(self, idx):
        """
        Atgriež datu punktu ar indeksu idx
        
        Parametri:
        idx (int): Datu punkta indekss
        
        Atgriež:
        tuple: (pazīmes, etiķete)
        """
        return self.features[idx], self.labels[idx]

# Definē klasi Model - pašu neironu tīklu
class IrisModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        """
        Inicializē neironu tīkla modeli
        
        Parametri:
        input_size (int): Ieejas slāņa izmērs (pazīmju skaits)
        hidden_size (int): Slēptā slāņa izmērs
        output_size (int): Izejas slāņa izmērs (klašu skaits)
        """
        super(IrisModel, self).__init__()
        self.layer1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.layer2 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        """
        Veic tiešo skrējienu caur neironu tīklu
        
        Parametri:
        x (torch.Tensor): Ieejas dati
        
        Atgriež:
        torch.Tensor: Izejas vērtības
        """
        x = self.layer1(x)
        x = self.relu(x)
        x = self.layer2(x)
        return x

# Globālie mainīgie
model = None
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class_names = ['Iris-setosa', 'Iris-versicolor', 'Iris-virginica']
label_to_idx = {'Iris-setosa': 0, 'Iris-versicolor': 1, 'Iris-virginica': 2}

# Datu sagatavošanas funkcija
def prepare_data(data_frame):
    """
    Sagatavo datus neironu tīkla apmācībai
    
    Parametri:
    data_frame (pandas.DataFrame): Pandas DataFrame ar datiem
    
    Atgriež:
    tuple: (IrisDataset, features, labels)
    """
    # Atdalam pazīmes un etiķetes
    features = data_frame.iloc[:, :-1].values
    species = data_frame.iloc[:, -1].values
    
    # Pārveidojam sugas nosaukumus par skaitliskiem indeksiem
    labels = np.array([label_to_idx[species_name] for species_name in species])
    
    # Pārveidojam masīvus par torch.Tensor
    features_tensor = torch.FloatTensor(features)
    labels_tensor = torch.LongTensor(labels)
    
    # Izveidojam DataSet objektu
    dataset = IrisDataset(features_tensor, labels_tensor)
    
    return dataset, features_tensor, labels_tensor

# Definē funkciju train, kuru izsaucot, modelis tiek trenēts ar treniņdatu kopu
def train():
    """
    Apmāca neironu tīkla modeli ar treniņa datiem
    """
    print("Training model...")
    global model
    
    # Ielādējam datus
    data_frame = pd.read_csv(TRAIN_DATA_URL)
    dataset, features, labels = prepare_data(data_frame)
    
    # Inicializējam modeli
    input_size = features.shape[1]  # pazīmju skaits
    hidden_size = 10  # slēptā slāņa izmērs
    output_size = len(class_names)  # klašu skaits
    
    model = IrisModel(input_size, hidden_size, output_size).to(device)
    
    # Definējam apmācības parametrus
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    batch_size = 8
    num_epochs = 100
    
    # Izveidojam DataLoader
    dataloader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True)
    
    # Apmācības cikls
    for epoch in range(num_epochs):
        for i, (features, labels) in enumerate(dataloader):
            features = features.to(device)
            labels = labels.to(device)
            
            # Tiešais skrējiens
            outputs = model(features)
            loss = criterion(outputs, labels)
            
            # Atpakaļpropagācija un optimizācija
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        # Izdrukājam zaudējumu katrus 10 soļus
        if (epoch + 1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
    
    print("Training completed!")

# Definē funkciju test, kuru izsaucot, modelis tiek pārbaudīts ar validācijas vai testa datu kopu
def test():
    """
    Pārbauda neironu tīkla modeli ar testa datiem
    """
    print("Testing model...")
    global model
    
    # Pārbaudam, vai modelis ir inicializēts
    if model is None:
        print("Error: Model not initialized!")
        return
    
    # Ielādējam testa datus
    data_url = TEST_DATA_URL
    
    data_frame = pd.read_csv(data_url)
    dataset, features, labels = prepare_data(data_frame)
    
    # Pārslēdzam modeli uz vērtēšanas režīmu
    model.eval()
    
    # Veicam prognozēšanu
    with torch.no_grad():
        outputs = model(features.to(device))
        _, predicted = torch.max(outputs.data, 1)
        
        correct = 0
        total = labels.size(0)
        
        # Izdrukājam rezultātus katram datu punktam
        for i in range(total):
            predicted_class = class_names[predicted[i].item()]
            actual_class = class_names[labels[i].item()]
            is_correct = predicted[i] == labels[i]
            
            if is_correct:
                correct += 1
            
            print(f"{i+1}. datu rindiņā klasifikatora rezultāts: {predicted_class}; reālā suga: {actual_class}, {'pareizi' if is_correct else 'nepareizi'}")
        
        # Izdrukājam kopējo precizitāti
        accuracy = 100 * correct / total
        print(f"Precizitāte: {accuracy:.0f}% ({correct}/{total})")

# Definē funkciju saveWeights, kuru izsaucot, tiek saglabāti modeļa svari
def saveWeights():
    """
    Saglabā neironu tīkla modeļa svarus
    """
    print("Saving weights...")
    global model
    
    # Pārbaudam, vai modelis ir inicializēts
    if model is None:
        print("Error: Model not initialized!")
        return
    
    # Saglabājam modeli
    torch.save(model.state_dict(), "iris_model.pth")
    print("Model weights saved to iris_model.pth")
    print("Upload the model to GitHub or another cloud service and update MODEL_URL")

# Definē funkciju loadWeights, kuru izsaucot, no URL MODEL_URL tiek ielādēti modeļa svari un piešķirti mūsu modelim
def loadWeights():
    """
    Ielādē neironu tīkla modeļa svarus no URL
    """
    print(f"Loading weights from {MODEL_URL}...")
    global model
    
    # Transformējam GitHub URL, lai iegūtu raw formātu
    raw_url = MODEL_URL.replace('github.com', 'raw.githubusercontent.com')
    raw_url = raw_url.replace('/blob/', '/')
    
    # Inicializējam modeli, ja tas vēl nav inicializēts
    if model is None:
        # Iegūstam pazīmju skaitu no testa datiem
        test_url = TEST_DATA_URL.replace('github.com', 'raw.githubusercontent.com')
        test_url = test_url.replace('/blob/', '/')
        data_frame = pd.read_csv(test_url)
        input_size = data_frame.iloc[:, :-1].shape[1]
        
        # Inicializējam modeli
        hidden_size = 10
        output_size = len(class_names)
        model = IrisModel(input_size, hidden_size, output_size).to(device)
    
    try:
        # Lejupielādējam modeļa svarus
        response = requests.get(raw_url)
        if response.status_code == 200:
            # Saglabājam svarus pagaidu failā
            with open("temp_model.pth", "wb") as f:
                f.write(response.content)
            
            # Ielādējam svarus modelī
            model.load_state_dict(torch.load("temp_model.pth", map_location=device))
            
            # Dzēšam pagaidu failu
            if os.path.exists("temp_model.pth"):
                os.remove("temp_model.pth")
            
            print("Model weights loaded successfully")
        else:
            print(f"Error downloading model weights: HTTP {response.status_code}")
    except Exception as e:
        print(f"Error loading model weights: {str(e)}")

### Datu ielādes bloks
Šajā koda blokā ielādē nepieciešamo datu kopa un sagatavo darbam ar modeli

In [4]:
# Pārveidojam GitHub URL uz raw formātu, lai varētu tieši ielādēt datus
if TRAIN == True:
    print("Ielādē treniņa datu kopu")
    # Pārveidojam GitHub URL, lai iegūtu raw formātu
    raw_url = TRAIN_DATA_URL.replace('github.com', 'raw.githubusercontent.com')
    raw_url = raw_url.replace('/blob/', '/')
    tabula = pd.read_csv(raw_url)
else:
    print("Ielādē testa datu kopu")
    # Pārveidojam GitHub URL, lai iegūtu raw formātu
    raw_url = TEST_DATA_URL.replace('github.com', 'raw.githubusercontent.com')
    raw_url = raw_url.replace('/blob/', '/')
    tabula = pd.read_csv(raw_url)

print(tabula.head())

Ielādē testa datu kopu
   SepalLengthCm  SepalWidthCm  PetalLengthCm  PetalWidthCm          Species
0            4.4           3.2            1.3           0.2      Iris-setosa
1            5.0           3.5            1.6           0.6      Iris-setosa
2            5.1           3.8            1.9           0.4      Iris-setosa
3            4.8           3.0            1.4           0.3      Iris-setosa
4            5.8           2.6            4.0           1.2  Iris-versicolor


### Modeļa apmācīšanas bloks
Šis bloks tiek palaists tikai tad, ja TRAIN == True.  
Satur tikai funkciju train.

In [5]:
if TRAIN == True:
    train()

### Modeļa saglabāšanas bloks
Šis bloks tiek palaists tikai tad, ja TRAIN == True.  
Satur tikai funkciju saveWeights.

In [6]:
if TRAIN == True:
    saveWeights()

### Modeļa ielādēšanas bloks
Šis bloks tiek palaists tikai tad, ja TRAIN == False.  
Satur tikai funkciju loadWeights.

In [7]:
if TRAIN == False:
    loadWeights()

Loading weights from https://raw.githubusercontent.com/achenlv/ap08426-random/refs/heads/main/iris_model.pth...
Model weights loaded successfully


### Modeļa pārbaudes kods
Šis bloks tiek palaists jebkurā gadījumā. Visiem iepriekšējiem blokiem jānostrādā pareizi atkarībā no TRAIN stāvokļa, lai šis kods paņemtu no jauna uztrenētu modeli un pārbaudīto to uz validācijas datu kopas, vai ielādētu saglabātu modeli un pārbaudītu to uz testa datu kopas.

Testa palaišanai jāizdrukā:
1. visu pārbaužu rezultāti viegli saprotamā formā (12 rindiņas, pa vienai katram validācijas kopas piemēram)
2. kopējais precizitātes novērtējums.
   
Paraugs iespējamam adekvātam noformējumam:

- 1\. datu rindiņā klasifikatora rezultāts: Iris-setosa; reālā suga: Iris-setosa, pareizi
- 2\. datu rindiņā klasifikatora rezultāts: Iris-versicolor; reālā suga: Iris-setosa, nepareizi
- ...
- 12\. datu rindiņā klasifikatora rezultāts: Iris-virginica; reālā suga: Iris-virginica, pareizi
    Precizitāte: 58% (7/12)


In [8]:
test()

Testing model...
1. datu rindiņā klasifikatora rezultāts: Iris-setosa; reālā suga: Iris-setosa, pareizi
2. datu rindiņā klasifikatora rezultāts: Iris-setosa; reālā suga: Iris-setosa, pareizi
3. datu rindiņā klasifikatora rezultāts: Iris-setosa; reālā suga: Iris-setosa, pareizi
4. datu rindiņā klasifikatora rezultāts: Iris-setosa; reālā suga: Iris-setosa, pareizi
5. datu rindiņā klasifikatora rezultāts: Iris-versicolor; reālā suga: Iris-versicolor, pareizi
6. datu rindiņā klasifikatora rezultāts: Iris-versicolor; reālā suga: Iris-versicolor, pareizi
7. datu rindiņā klasifikatora rezultāts: Iris-versicolor; reālā suga: Iris-versicolor, pareizi
8. datu rindiņā klasifikatora rezultāts: Iris-versicolor; reālā suga: Iris-versicolor, pareizi
9. datu rindiņā klasifikatora rezultāts: Iris-virginica; reālā suga: Iris-virginica, pareizi
10. datu rindiņā klasifikatora rezultāts: Iris-virginica; reālā suga: Iris-virginica, pareizi
11. datu rindiņā klasifikatora rezultāts: Iris-virginica; reālā suga